<a href="https://colab.research.google.com/github/Adarsh-hello/deepfake_detection/blob/main/deepfake_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install streamlit pyngrok gdown


In [15]:
%%writefile app.py
import streamlit as st
import numpy as np
from PIL import Image
import os


try:
    import gdown
except ImportError:
    os.system('pip install gdown')
    import gdown

from tensorflow.keras.models import load_model


st.set_page_config(page_title="Deepfake Detection using CNN", page_icon="🧠", layout="centered")
st.title("🧠 Deepfake Detection - CNN Model (with Google Drive)")


MODEL_PATH = "Cnn_model.h5"
DRIVE_FILE_ID = "1glEUFHmDxCFHWSRfPFS0BWxdk9jEMpJG"


def download_model():
    if not os.path.exists(MODEL_PATH):
        url = f"https://drive.google.com/uc?id={DRIVE_FILE_ID}"
        gdown.download(url, MODEL_PATH, quiet=False)


uploaded_file = st.file_uploader("📤 Upload an image", type=["jpg", "jpeg", "png"])


@st.cache_resource
def load_cnn_model():
    download_model()
    cnn = load_model(MODEL_PATH)
    return cnn

try:
    cnn_model = load_cnn_model()
except Exception as e:
    st.error(f"❌ Error loading model: {e}")
    st.stop()


labels = ['Real', 'Fake']


def preprocess(img):
    img = img.resize((128, 128))
    img_array = np.array(img) / 255.0
    if img_array.shape[-1] == 4:
        img_array = img_array[..., :3]
    return np.expand_dims(img_array, axis=0)


if uploaded_file is not None:
    st.image(uploaded_file, caption="🖼 Uploaded Image", use_column_width=False, width=250)
    try:
        image = Image.open(uploaded_file)
        processed = preprocess(image)
        cnn_pred = cnn_model.predict(processed)[0]
        cnn_result = labels[np.argmax(cnn_pred)]

        st.markdown("### 🔍 Prediction Result:")
        st.success(f"🧠 CNN Prediction: {cnn_result} ({cnn_pred[np.argmax(cnn_pred)]*100:.2f}%)")

    except Exception as e:
        st.error(f"Prediction error: {e}")


Overwriting app.py


In [16]:
!pkill -f ngrok


In [19]:
from pyngrok import ngrok
import time
import threading


def start_streamlit():
    !streamlit run app.py

thread = threading.Thread(target=start_streamlit)
thread.start()


time.sleep(5)


ngrok.set_auth_token("2wJv717kfaZlPwHjKAAQMErod1t_5iYEcD4RaSpoLkHAwgz2i")



public_url = ngrok.connect(8501)
print(public_url)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.197.89.31:8501

NgrokTunnel: "https://635d-104-197-89-31.ngrok-free.app" -> "http://localhost:8501"
